# To do list
- switch to storage
- ingest csvs like storage class
- figure out traversals

In [30]:
import knowledge_graph as g
import pandas as pd
from dotenv import load_dotenv, find_dotenv
import os
from storage import Metadata, Storage

In [31]:
load_dotenv(find_dotenv())

True

In [32]:
graph = g.Graph(os.getenv('GRAPH_URI'), os.getenv('GRAPH_USER') , os.getenv('GRAPH_PASS'))


In [33]:
csv_files = os.listdir("../../dev/data/Football Example/")

In [34]:
csv_files[0]

'appearances.csv'

In [35]:
# m = metadata("../../dev/data/Football Example/" + csv_files[0])
# "../../dev/data/Football Example/apperances.csv"

# Pull all test CSVs
- pull all test dfs into a dict/hash map, pull their columns
- pull their PK/FK pairs
  - all PKs are first column
  - check to see if FK appears in column list
  - write another check to find all PK/FKs in dataframes based on keywords ('ID', 'KEY')
    - check across all dataframes for matching keys

- example
  - dataframe w/ GameID
  - dataframe w/ PlayerID
  - dataframe w/ GameID, PlayerID, etc
    - this would be composite key if multiple IDs
  - dataframe w/ GameID (FK), PlayerID (FK), EventID (PK)
    - has one PK, how would we identify?
    - would it be same as node name + id/key?
- non-automated example
  - make a file that makes PK/FK pairs w/ table names
    - relationships.txt
    - relationships.json
- 

In [36]:
df_dict={}
for f in csv_files:
      
    # read the csv file
    df = pd.read_csv("../../dev/data/Football Example/" + f, encoding='latin-1')
    df_name = f.split("\\")[-1].split('.csv')[0]
    df_keys = []
    for column in df.columns.tolist():
        if ("id") in column.lower():
            df_keys.append(column)
            
    df_dict[df_name] = {"columns" : df.columns.tolist(),
                        "keys": df_keys}
                        
      
    # print the location and filename
    print('Location:', f)
    print('File Name:', f.split("\\")[-1].split('.csv')[0])
      
    # print the content
    # print('Content:')
    # display(df)
    # print()

Location: appearances.csv
File Name: appearances
Location: games.csv
File Name: games
Location: leagues.csv
File Name: leagues
Location: players.csv
File Name: players
Location: shots.csv
File Name: shots
Location: teams.csv
File Name: teams
Location: teamstats.csv
File Name: teamstats


# Turn Data Frames into Nodes
- assuming after turning all file types/databases into pandas dataframes we can turn them into nodes
  - also assuming we might not even have to pull full DFs, just metadata, columns, etc.

In [37]:
graph.delete_all()

In [38]:
df_dict.items()

dict_items([('appearances', {'columns': ['gameID', 'playerID', 'goals', 'ownGoals', 'shots', 'xGoals', 'xGoalsChain', 'xGoalsBuildup', 'assists', 'keyPasses', 'xAssists', 'position', 'positionOrder', 'yellowCard', 'redCard', 'time', 'substituteIn', 'substituteOut', 'leagueID'], 'keys': ['gameID', 'playerID', 'leagueID']}), ('games', {'columns': ['gameID', 'leagueID', 'season', 'date', 'homeTeamID', 'awayTeamID', 'homeGoals', 'awayGoals', 'homeProbability', 'drawProbability', 'awayProbability', 'homeGoalsHalfTime', 'awayGoalsHalfTime', 'B365H', 'B365D', 'B365A', 'BWH', 'BWD', 'BWA', 'IWH', 'IWD', 'IWA', 'PSH', 'PSD', 'PSA', 'WHH', 'WHD', 'WHA', 'VCH', 'VCD', 'VCA', 'PSCH', 'PSCD', 'PSCA'], 'keys': ['gameID', 'leagueID', 'homeTeamID', 'awayTeamID']}), ('leagues', {'columns': ['leagueID', 'name', 'understatNotation'], 'keys': ['leagueID']}), ('players', {'columns': ['playerID', 'name'], 'keys': ['playerID']}), ('shots', {'columns': ['gameID', 'shooterID', 'assisterID', 'minute', 'situatio

In [39]:
for name, dict in df_dict.items():
    print(name, dict.get("keys"))
    keys = dict.get("keys")
    print(name, dict.get("columns"))
    columns = dict.get("columns")
    print("Creating node!!!")
    graph.add_table_node(name, dict.get("keys"), dict.get("columns"), "TABLE")

appearances ['gameID', 'playerID', 'leagueID']
appearances ['gameID', 'playerID', 'goals', 'ownGoals', 'shots', 'xGoals', 'xGoalsChain', 'xGoalsBuildup', 'assists', 'keyPasses', 'xAssists', 'position', 'positionOrder', 'yellowCard', 'redCard', 'time', 'substituteIn', 'substituteOut', 'leagueID']
Creating node!!!
games ['gameID', 'leagueID', 'homeTeamID', 'awayTeamID']
games ['gameID', 'leagueID', 'season', 'date', 'homeTeamID', 'awayTeamID', 'homeGoals', 'awayGoals', 'homeProbability', 'drawProbability', 'awayProbability', 'homeGoalsHalfTime', 'awayGoalsHalfTime', 'B365H', 'B365D', 'B365A', 'BWH', 'BWD', 'BWA', 'IWH', 'IWD', 'IWA', 'PSH', 'PSD', 'PSA', 'WHH', 'WHD', 'WHA', 'VCH', 'VCD', 'VCA', 'PSCH', 'PSCD', 'PSCA']
Creating node!!!
leagues ['leagueID']
leagues ['leagueID', 'name', 'understatNotation']
Creating node!!!
players ['playerID']
players ['playerID', 'name']
Creating node!!!
shots ['gameID', 'shooterID', 'assisterID']
shots ['gameID', 'shooterID', 'assisterID', 'minute', 'si

In [40]:
table_relationships = []
for table1, dict1 in df_dict.items():
    keys1 = dict1.get("keys")
    for table2, dict2 in df_dict.items():
        keys2 = dict2.get("keys")
        if table1 != table2:
            shared_keys = list(set(keys1) & set(keys2))
            if shared_keys:
                for i, shared_key in enumerate(shared_keys, start = 1):
                    table_relationships.append((table1, table2, f"{table1}.{shared_key} = {table2}.{shared_key}", i))

In [41]:
# These relationships weren't recognized, as shooterID and assisterID doesn't explicely say person, bet contextually we know shooters and assisters are people
table1, table2 = 'shots', 'players'
table_relationships.append((table1, table2, f"{table1}.shooterID = {table2}.playerID", i)) 
table_relationships.append((table1, table2, f"{table1}.assisterID = {table2}.playerID", i))

In [42]:
table_relationships


[('appearances', 'games', 'appearances.gameID = games.gameID', 1),
 ('appearances', 'games', 'appearances.leagueID = games.leagueID', 2),
 ('appearances', 'leagues', 'appearances.leagueID = leagues.leagueID', 1),
 ('appearances', 'players', 'appearances.playerID = players.playerID', 1),
 ('appearances', 'shots', 'appearances.gameID = shots.gameID', 1),
 ('appearances', 'teamstats', 'appearances.gameID = teamstats.gameID', 1),
 ('games', 'appearances', 'games.gameID = appearances.gameID', 1),
 ('games', 'appearances', 'games.leagueID = appearances.leagueID', 2),
 ('games', 'leagues', 'games.leagueID = leagues.leagueID', 1),
 ('games', 'shots', 'games.gameID = shots.gameID', 1),
 ('games', 'teamstats', 'games.gameID = teamstats.gameID', 1),
 ('leagues', 'appearances', 'leagues.leagueID = appearances.leagueID', 1),
 ('leagues', 'games', 'leagues.leagueID = games.leagueID', 1),
 ('players', 'appearances', 'players.playerID = appearances.playerID', 1),
 ('shots', 'appearances', 'shots.gameI

In [43]:
for relationship in table_relationships:
    node_one = relationship[0]
    node_two = relationship[1] 
    key = f"""JOIN {{key : "{relationship[2]}"}}"""
    # print(node_one, node_two, key)
    graph.add_relationship(name_node_one= node_one, name_node_two= node_two, relation_name= key)

In [44]:
result = graph.node_traversal(name_node_one="games", name_node_two="teams", max_traversal=3)

ResultConsumedError: The result is out of scope. The associated transaction has been closed. Results can only be used while the transaction is open.

In [21]:
result

In [18]:
for record in result:
    path = record["path"]
    # Process the path or extract specific data as needed
    print(path)

ResultConsumedError: The result is out of scope. The associated transaction has been closed. Results can only be used while the transaction is open.